# News data category

In [10]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Flatten, Concatenate, Input, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd

In [5]:
news_data = pd.read_json('./data/News_Category_Dataset_v2.json', lines=True)

news_data = news_data.loc[:, ['category', 'headline']]

In [6]:
pd.factorize(news_data['category'])

(array([ 0,  1,  1, ..., 10, 10, 10], dtype=int64),
 Index(['CRIME', 'ENTERTAINMENT', 'WORLD NEWS', 'IMPACT', 'POLITICS',
        'WEIRD NEWS', 'BLACK VOICES', 'WOMEN', 'COMEDY', 'QUEER VOICES',
        'SPORTS', 'BUSINESS', 'TRAVEL', 'MEDIA', 'TECH', 'RELIGION', 'SCIENCE',
        'LATINO VOICES', 'EDUCATION', 'COLLEGE', 'PARENTS', 'ARTS & CULTURE',
        'STYLE', 'GREEN', 'TASTE', 'HEALTHY LIVING', 'THE WORLDPOST',
        'GOOD NEWS', 'WORLDPOST', 'FIFTY', 'ARTS', 'WELLNESS', 'PARENTING',
        'HOME & LIVING', 'STYLE & BEAUTY', 'DIVORCE', 'WEDDINGS',
        'FOOD & DRINK', 'MONEY', 'ENVIRONMENT', 'CULTURE & ARTS'],
       dtype='object'))

In [7]:
category_list = pd.factorize(news_data['category'])[1]

In [9]:
news_data['category'] = pd.factorize(news_data['category'])[0]
news_data

,category,headline
0,0,There Were 2 Mass Shootings In Texas Last Week...
1,1,Will Smith Joins Diplo And Nicky Jam For The 2...
2,1,Hugh Grant Marries For The First Time At Age 57
3,1,Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4,1,Julianna Margulies Uses Donald Trump Poop Bags...
...,...,...
200848,14,RIM CEO Thorsten Heins' 'Significant' Plans Fo...
200849,10,Maria Sharapova Stunned By Victoria Azarenka I...
200850,10,"Giants Over Patriots, Jets Over Colts Among M..."
200851,10,Aldon Smith Arrested: 49ers Linebacker Busted ...


In [11]:
news_data['headline'] = news_data['headline'].str.replace("[^\w]", " ")

news_train, news_test, y_train, y_test = train_test_split(news_data['headline'], news_data['category'], random_state=123)

In [12]:
# 원 핫 벡터
# y_train = to_categorical(y_train, num_classes=) # 다른 방법
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

len(y_train[0]), len(y_test[0])

(41, 41)

In [13]:
stopwords = ['a', 'an']

X_train = []
for stc in news_train:
    token = []
    for word in stc.split():
        if word not in stopwords:
            token.append(word)
    X_train.append(token)

X_test = []
for stc in news_test:
    token = []
    for word in stc.split():
        if word not in stopwords:
            token.append(word)
    X_test.append(token)


In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(25000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [15]:
print(len(tokenizer.word_index))

low_count = 0
for word, word_count in tokenizer.word_counts.items():
    if word_count == 1:
        low_count += 1
print(low_count)

49704
20028


In [16]:
max_length = 0
for data in X_train:
    if max_length < len(data):
        max_length = len(data)
print(max_length)

39


In [17]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 15
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [18]:
model = Sequential()
model.add(Embedding(25000, 128))
model.add(LSTM(128))
model.add(Dense(41, activation='sigmoid'))

In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1)

Train on 150639 samples, validate on 50214 samples
150639/150639 [==============================] - 352s 2ms/sample - loss: 1.5784 - acc: 0.5684 - val_loss: 1.5208 - val_acc: 0.5811


In [22]:
sentence = "rice"

token_stc = sentence.split()
encode_stc = tokenizer.texts_to_sequences([token_stc])
pad_stc = pad_sequences(encode_stc, max_len=15)

score = model.predict(pad_stc)
print(score)

TypeError: pad_sequences() got an unexpected keyword argument 'max_len'

In [21]:
print(category_list[score.argmax()], score[0, score.argmax()])

NameError: name 'score' is not defined